In [21]:
from bs4 import BeautifulSoup
from selenium import webdriver
import requests
import pandas as pd
import numpy as np
import time
from math import floor
import pickle

In [10]:
def Digits(a):
    digit=0
    while a>1:
        digit+=1
        a=a/10
    return digit

In [11]:
def lastPageLinks(a):
    if a<100:
        return a%50
    b=Digits(a)
    c=(a%(10**(b-1)))%50
    return c

In [18]:
def generateDataPerPage(imdbDict,movies):
    details=[]
    for movie in movies:

        try:
            imdbDict['IMDB Rating'].append(movie.find('div',class_='inline-block ratings-imdb-rating').strong.text.strip())
        except:
            imdbDict['IMDB Rating'].append(None)
            
#         print(movie.find('span',class_='runtime').text.strip())
        try:
            imdbDict['Total Duration (in Minutes)'].append(float(movie.find('span',class_='runtime').text.strip().split()[0]))
        except:
            imdbDict['Total Duration (in Minutes)'].append(None)
            
        try:
            imdbDict['Votes'].append(float(movie.find('span',{'name':'nv'}).text.replace(',','')))
        except:
            imdbDict['Votes'].append(None)
            
        try:
            imdbDict['Description'].append(movie.find_all('p',class_='text-muted')[1].text.strip())
        except:
            imdbDict['Description'].append(None)
            
        try:
            imdbDict["Movie name"].append(movie.a.text.strip())
        except:
            imdbDict['Movie name'].append(None)
            
        try:
            imdbDict['Metascore'].append(float(movie.find('span',class_='metascore favorable').text.strip()))
        except:
            imdbDict['Metascore'].append(None)
            
        try:
            details.append(movie.find('p',class_='').text)
        except:
            details.append(None)

    return imdbDict

In [13]:
def generateMovieLinks(movie_limit):
    print('Generating movie links')
    links=[]
    b=floor((movie_limit-1)/50)
    c=50*b+1
    pages=len(range(0,c,50))
    for i in range(0,c,50):
#         print(i+1)
        links.append(f'https://www.imdb.com/search/title/?release_date=2020-01-01,2020-12-31&sort=num_votes,desc&start={i+1}&ref_=adv_nxt')
    return pages,links
pages,links=generateMovieLinks(51)
print(pages,links)

Generating movie links
2 ['https://www.imdb.com/search/title/?release_date=2020-01-01,2020-12-31&sort=num_votes,desc&start=1&ref_=adv_nxt', 'https://www.imdb.com/search/title/?release_date=2020-01-01,2020-12-31&sort=num_votes,desc&start=51&ref_=adv_nxt']


In [14]:
def imDB(movie_limit=1000,toCSV=False):
    movie_limit=min(10000,movie_limit)
    pages,links=generateMovieLinks(movie_limit)
    lastLimit=lastPageLinks(movie_limit)
#     print(linksPerPage)
    print(f'{pages} links generated')
    features=["Movie name","IMDB Rating","Description","Votes","Metascore","Total Duration (in Minutes)"]
    imdbDict={key:[] for key in features}
    for index,link in enumerate(links):
        print(f'Working on link no: {index+1}')
        html_text = requests.get(link).text
        soup=BeautifulSoup(html_text)
        movies=soup.find_all('div',class_='lister-item-content')
#         print(len(movies))
        if index == pages-1 and movie_limit%50 > 0:
            imdbDict=generateDataPerPage(imdbDict,movies[:lastLimit])
        else:
            imdbDict=generateDataPerPage(imdbDict,movies)
        
    df=pd.DataFrame(imdbDict)
    if toCSV == True:
        df.to_csv('Released between 2020-01-01 and 2020-12-31 (Sorted by Number of Votes Descending).csv',index=True)
    
    return df,imdbDict

In [22]:
%%time
df0,imdbDict0=imDB(10000,True)

Generating movie links
200 links generated
Working on link no: 1
Working on link no: 2
Working on link no: 3
Working on link no: 4
Working on link no: 5
Working on link no: 6
Working on link no: 7
Working on link no: 8
Working on link no: 9
Working on link no: 10
Working on link no: 11
Working on link no: 12
Working on link no: 13
Working on link no: 14
Working on link no: 15
Working on link no: 16
Working on link no: 17
Working on link no: 18
Working on link no: 19
Working on link no: 20
Working on link no: 21
Working on link no: 22
Working on link no: 23
Working on link no: 24
Working on link no: 25
Working on link no: 26
Working on link no: 27
Working on link no: 28
Working on link no: 29
Working on link no: 30
Working on link no: 31
Working on link no: 32
Working on link no: 33
Working on link no: 34
Working on link no: 35
Working on link no: 36
Working on link no: 37
Working on link no: 38
Working on link no: 39
Working on link no: 40
Working on link no: 41
Working on link no: 42


In [23]:
df0

,Movie name,IMDB Rating,Description,Votes,Metascore,Total Duration (in Minutes)
0,Tenet,7.4,"Armed with only one word, Tenet, and fighting ...",447164.0,69.0,150.0
1,The Queen's Gambit,8.6,"Orphaned at the tender age of nine, prodigious...",380817.0,NaN,395.0
2,Soul,8.1,"After landing the gig of a lifetime, a New Yor...",294048.0,83.0,100.0
3,Wonder Woman 1984,5.4,Diana must contend with a work colleague and b...,244864.0,NaN,151.0
4,Birds of Prey: And the Fantabulous Emancipatio...,6.1,"After splitting with the Joker, Harley Quinn j...",219534.0,NaN,109.0
...,...,...,...,...,...,...
9995,Josiah,8.9,Josiah is a story of a young man who falls int...,124.0,NaN,95.0
9996,Lenox Hill,8.3,"Two neurosurgeons, an ER doctor and an OB-GYN ...",124.0,NaN,NaN
9997,50 States of Fright,6.3,Miami police respond to an odd call about a st...,124.0,NaN,5.0
9998,Apodrasi apo ton Efthrafsto Planiti,7.0,"A boy-meets-boy tale, a few hours before the w...",124.0,NaN,17.0


In [24]:
imdbDict0

{'Movie name': ['Tenet',
  "The Queen's Gambit",
  'Soul',
  'Wonder Woman 1984',
  'Birds of Prey: And the Fantabulous Emancipation of One Harley Quinn',
  'The Invisible Man',
  'A Quiet Place Part II',
  'Extraction',
  'The Trial of the Chicago 7',
  'Ted Lasso',
  'Enola Holmes',
  'The Old Guard',
  'Bad Boys for Life',
  'Promising Young Woman',
  'Palm Springs',
  'Nomadland',
  'Mulan',
  'Scam 1992: The Harshad Mehta Story',
  'Borat Subsequent Moviefilm: Delivery of Prodigious Bribe to American Regime for Make Benefit Once Glorious Nation of Kazakhstan',
  'Onward',
  'Druk',
  'Dil Bechara',
  'I Care a Lot',
  'The Devil All the Time',
  'The Father',
  'Love and Monsters',
  'Sonic the Hedgehog',
  'Soorarai Pottru',
  'Greenland',
  'The Last Dance',
  'The Hunt',
  'Bridgerton',
  'Sadak 2',
  'The Haunting of Bly Manor',
  'Eurovision Song Contest: The Story of Fire Saga',
  'Greyhound',
  'The Undoing',
  'Spenser Confidential',
  'Project Power',
  'News of the World

In [35]:
with open('imdb_movies','wb') as file:
    pickle.dump(df0,file)
    file.close()

In [34]:
with open('imdb_movies','rb') as file:
    df=pickle.load(file)
    file.close()